# Normalization of LIME image explanations

Normalize explanations of trained models ('model_names') on 100 samples per class from a set of datasets ('dataset_paths'). 

- Trained model weights loaded from 'models_save_dir'. 

- Evaluation results in 'results_dir' are used to select 100 positive samples of every class.

- Explanations are loaded from subfolders under 'results_dir'.

- Normalized explanations are stored in subfolders under 'results_dir'.

In [58]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import dlib
from PIL import Image

from ekman_expressions.nets import getNetByName
from ekman_expressions.normalization import transform_mask

In [59]:
# Models
model_names = ['SilNet', 'WeiNet', 'AlexNet', 'SongNet', 'InceptionV3',
               'VGG19', 'VGG16', 'ResNet50', 'ResNet101V2', 'Xception',
               'MobileNetV3Large', 'EfficientNetV2B0']

# Number of k-cross validations and folder where they are located
# Alternatively set the paths to the target training and test manually
K = 5
dataset_paths_root = '../datasets/'
dataset_paths_train = []
dataset_paths_test = []
for i in range(K):
    dataset_paths_train.append(dataset_paths_root + 'CV' + str(i+1))
    dataset_paths_test.append(dataset_paths_root + 'CV' + str(i+1) + '_test')

# Folder where the trained models were saved
models_save_dir = '../models'

# Folder where the evaluation results are saved
results_dir = '../results'

# Labels of the classes
label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise']

# Path to the predictor file
predictor_path = '../resources/shape_predictor_68_face_landmarks.dat'

# Dimensions of normalized images
norm_width = 224
norm_height = 275
vertical_space = 25

## Transform LIME masks and images

In [ ]:
# Init progress bar
progress = tqdm(total=len(model_names)*len(dataset_paths_train)*len(label_names)*100)

# Load predictor
predictor = dlib.shape_predictor(predictor_path)

# Iterate over each net
for model_name in model_names:
        
    # Load model img size
    _, img_size = getNetByName(model_name)
    
    # Iterate over each CV set
    for train_path, test_path in zip(dataset_paths_train, dataset_paths_test):

        # Results foder
        results_net = os.path.join(results_dir, model_name + '_'
                                   + os.path.basename(train_path)
                                   + '_results')

        # 100 positives per class folder
        imgs100 = os.path.join(results_net, 'imgs_100')

        # Iterate over each class
        for class_dir in os.listdir(imgs100):

            # Path of imgs folder
            img_path = os.path.join(imgs100, class_dir, 'positives')

            # Path of LIME masks
            exp_mask_path = os.path.join(imgs100, class_dir, 'lime_masks')

            # Path of imgs with landmarks
            positives_landmarks_path = os.path.join(imgs100, class_dir, 'positives_landmarks')
            if not os.path.exists(positives_landmarks_path):
                os.mkdir(positives_landmarks_path)

            # Path of LIME masks with landmarks
            lime_masks_landmarks_path = os.path.join(imgs100, class_dir, 'lime_masks_landmarks')
            if not os.path.exists(lime_masks_landmarks_path):
                os.mkdir(lime_masks_landmarks_path)

            # Path of transformed imgs
            positives_transformed_path = os.path.join(imgs100, class_dir, 'positives_transformed')
            if not os.path.exists(positives_transformed_path):
                os.mkdir(positives_transformed_path)

            # Path of transformed LIME masks
            lime_masks_transformed_path = os.path.join(imgs100, class_dir, 'lime_masks_transformed')
            if not os.path.exists(lime_masks_transformed_path):
                os.mkdir(lime_masks_transformed_path)

            # Transform each img and mask
            for img_name, mask_name in zip(os.listdir(img_path), os.listdir(exp_mask_path)):
                
                # Load image and LIME mask
                img = Image.open(os.path.join(img_path, img_name))
                img = img.resize((img_size, img_size))
                img = np.array(img.convert("RGB"))
                
                mask = cv2.imread(os.path.join(exp_mask_path, mask_name))
            
                # Draw landmarks on img and LIME mask, and transform both
                img_land, lime_land, img_trans, lime_trans = transform_mask(
                    predictor=predictor,
                    maxx=norm_width,
                    maxy=norm_height,
                    vertical_space=vertical_space,
                    image_cropped=img,
                    image_lime=mask)
                
                # Save images
                cv2.imwrite(os.path.join(positives_landmarks_path, img_name[:-4] + '_landmarks.png'), img_land)
                cv2.imwrite(os.path.join(lime_masks_landmarks_path, img_name[:-4] + '_mask_landmarks.png'), lime_land)
                cv2.imwrite(os.path.join(positives_transformed_path, img_name[:-4] + '_transformed.png'), img_trans)
                cv2.imwrite(os.path.join(lime_masks_transformed_path, img_name[:-4] + '_mask_transformed.png'), lime_trans)

                # Update progress
                progress.update(1)

# Close progress
progress.close()